# Import packages

In [1]:
import pandas as pd
import numpy as np
import cv2
import math
import os
import glob

In [2]:
#directory where behavior videos are located
video_directory = '01_Raw_videos'
Adult_model_directory = '02_Adult_model'
Infant_model_adjusted_directory = '04_Infants_model_adjusted'
Nest_model_directory = '05_Nest_model'
output_directory = '06_Unified_dataframe'
file_location = os.path.join(video_directory, '*.mp4')

# create list of filenames for every video to be processed
# this list is the input for the video processing function
filenames = glob.glob(file_location)
videoname = []
for f in filenames:
    
    # establish name for output file from the input filename
    x = os.path.split(f)
    x = x[1].split('.mp4')
    x = x[0]
    videoname.append(x)
videoname

['Gal_35_father_retreive_trial_1', 'Gal_38_mother_retreive_trial_1']

# Import and combine data from DeepLabCut

In [3]:
for x in videoname:
    #Import the H5 file for Infants
    Infant_file = os.path.join(Infant_model_adjusted_directory, x)
    Infant_var = glob.glob(Infant_file + '*')
    Infants = pd.read_hdf(Infant_var[0])
    #Infants = Infants.droplevel(level=0, axis=1)
    Infants = Infants.apply(pd.to_numeric)
    Infant_bodyparts = Infants.columns.get_level_values(level=1)
    Infant_bodyparts = set(Infant_bodyparts.tolist())
    
    #Import the H5 file for Adults
    Adult_file = os.path.join(Adult_model_directory, x)
    Adult_var = glob.glob(Adult_file + '*')
    Adults = pd.read_hdf(Adult_var[0])
    Adults = Adults.droplevel(level=0, axis=1)
    Adults = Adults.apply(pd.to_numeric)
    # The Adult DLC dataframe was created using single animal DLC so there is no multiindex level cooresponding to 'individuals'
    # Must make the indexes match before we can merge.
    Adults['individuals'] = 'Adult'
    Adults = Adults.set_index('individuals', append=True).unstack('individuals')
    Adults = Adults.reorder_levels([2,0,1], axis=1)
    Adult_bodyparts = Adults.columns.get_level_values(level=1)
    Adult_bodyparts = set(Adult_bodyparts.tolist())
    
    #Import the H5 file for Nests
    Nest_file = os.path.join(Nest_model_directory, x)
    Nest_var = glob.glob(Nest_file + '*')
    Nest = pd.read_hdf(Nest_var[0])
    # Determine the maxium amount of bounding boxes present in a single frame
    # This will 
    nest_columns = Nest.columns

    nest_list = set()
    for c in nest_columns:
        nest_number = str([c])
        nest_number = nest_number[7]
        nest_list.add(nest_number)
    
    #Tidy the nominclature
    for b in nest_list:
        nest_number = b
        if 'nest_{}_0'.format(b) in nest_columns:
            Nest['nest_{}_0'.format(b)] = Nest['nest_{}_0'.format(b)].combine_first(Nest['nest_{}'.format(b)])
        else:
            Nest['nest_{}_0'.format(b)] = Nest['nest_{}'.format(b)]
        Nest.drop(['nest_{}'.format(b)], axis=1, inplace=True)

    #NEST['Frame'] = np.arange(len(NEST))    
    Nest = Nest.reindex(sorted(Nest.columns), axis=1)

    # Create a list to be used when making the MultiIndex#
    # Cooresponds to the bounding box that the polygon belongs to
    bounding_box_number = []
    for a in Nest.columns:
        test = a[5]
        bounding_box_number.append(test)

    index = []
    for z in Nest.columns:
        var = ('Nest', z[5], z,)
        index.append(var)
    
    multi = pd.MultiIndex.from_tuples(index, names=['individuals', 'bodyparts', 'coords'])

    Nest.columns = multi
    
    # Combine all three dataframes
    Full = pd.concat([Adults, Infants, Nest], axis=1)

    # Save output to HDf5
    h5File = (x + "." +'h5')
    h5File = os.path.join(output_directory, h5File)
    Full.to_hdf(h5File, key="NEST", mode='w')

C:\Users\wrigh\anaconda3\lib\site-packages\pandas\core\generic.py:2490: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->MultiIndex([('Nest', '0', 'nest_0_0'),
            ('Nest', '0', 'nest_0_1'),
            ('Nest', '0', 'nest_0_2'),
            ('Nest', '1', 'nest_1_0'),
            ('Nest', '2', 'nest_2_0')],
           names=['individuals', 'bodyparts', 'coords'])]

  pytables.to_hdf(
C:\Users\wrigh\anaconda3\lib\site-packages\pandas\core\generic.py:2490: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->MultiIndex([('Nest', '0', 'nest_0_0'),
            ('Nest', '0', 'nest_0_1'),
            ('Nest', '0', 'nest_0_2'),
            ('Nest', '1', 'nest_1_0')],
           names=['individuals', 'bodyparts', 'coords'])]

  pytables.to_hd

In [8]:
Nest

individuals Infant1                                                   \
bodyparts      head                    middle_head                     
coords            x       y likelihood           x      y likelihood   
0               NaN     NaN        NaN         NaN    NaN        NaN   
1               NaN     NaN        NaN         NaN    NaN        NaN   
2               NaN     NaN        NaN         NaN    NaN        NaN   
3               NaN     NaN        NaN         NaN    NaN        NaN   
4               NaN     NaN        NaN         NaN    NaN        NaN   
...             ...     ...        ...         ...    ...        ...   
13951         176.0  169.25   0.010002     166.875  164.0   0.010002   
13952         176.0  169.25   0.010002     166.875  164.0   0.010002   
13953         176.0  169.25   0.010002     166.875  164.0   0.010002   
13954         176.0  169.25   0.010002     166.875  164.0   0.010002   
13955         176.0  169.25   0.010002     166.875  164.0   0.010002   

individuals                                         ...    Infant2  \
bodyparts   middle_tail                       tail  ...       head   
coords                x        y likelihood      x  ... likelihood   
0                   NaN      NaN        NaN    NaN  ...        NaN   
1                   NaN      NaN        NaN    NaN  ...        NaN   
2                   NaN      NaN        NaN    NaN  ...        NaN   
3                   NaN      NaN        NaN    NaN  ...        NaN   
4                   NaN      NaN        NaN    NaN  ...        NaN   
...                 ...      ...        ...    ...  ...        ...   
13951             164.0  161.625   0.010002  164.5  ...   0.010002   
13952             164.0  161.625   0.010002  164.5  ...   0.010002   
13953             164.0  161.625   0.010002  164.5  ...   0.010002   
13954             164.0  161.625   0.010002  164.5  ...   0.010002   
13955             164.0  161.625   0.010002  164.5  ...   0.010002   

individuals                                                                 \
bodyparts   middle_head                      middle_tail                     
coords                x         y likelihood           x      y likelihood   
0                   NaN       NaN        NaN         NaN    NaN        NaN   
1                   NaN       NaN        NaN         NaN    NaN        NaN   
2                   NaN       NaN        NaN         NaN    NaN        NaN   
3                   NaN       NaN        NaN         NaN    NaN        NaN   
4                   NaN       NaN        NaN         NaN    NaN        NaN   
...                 ...       ...        ...         ...    ...        ...   
13951            145.25  105.0625   0.010002     111.125  159.0   0.010002   
13952            145.25  105.0625   0.010002     111.125  159.0   0.010002   
13953            145.25  105.0625   0.010002     111.125  159.0   0.010002   
13954            145.25  105.0625   0.010002     111.125  159.0   0.010002   
13955            145.25  105.0625   0.010002     111.125  159.0   0.010002   

individuals                               
bodyparts       tail                      
coords             x        y likelihood  
0                NaN      NaN        NaN  
1                NaN      NaN        NaN  
2                NaN      NaN        NaN  
3                NaN      NaN        NaN  
4                NaN      NaN        NaN  
...              ...      ...        ...  
13951        112.875  167.875   0.010002  
13952        112.875  167.875   0.010002  
13953        112.875  167.875   0.010002  
13954        112.875  167.875   0.010002  
13955        112.875  167.875   0.010002  

[13956 rows x 24 columns]